# Introduction to connecting and Querying the Augur DB

## Connect to your database

Until the Operate First enviroment can connect to the DB, use config file to access. Do not push config file to Github repo

In [1]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import os

with open("../comm_cage.json") as config_file:
    config = json.load(config_file)

In [2]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [3]:
aval_repos = pd.DataFrame()
repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived,
                b.repo_git
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id
            ORDER BY
                rg_name,
                repo_name;
    """)
aval_repos = pd.read_sql(repo_query, con=engine)
display(aval_repos)
aval_repos.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived,repo_git
0,3scale,25556,3scale,33134,Parent not available,0.0,https://github.com/3scale/3scale.github.io
1,3scale,25556,3scale-amp-openshift-templates,33126,Parent not available,0.0,https://github.com/3scale/3scale-amp-openshift...
2,3scale,25556,3scale-api-python,33130,3scale-qe/3scale-api-python,0.0,https://github.com/3scale/3scale-api-python
3,3scale,25556,3scale-api-ruby,33158,Parent not available,0.0,https://github.com/3scale/3scale-api-ruby
4,3scale,25556,3scale-go-client,33169,Parent not available,0.0,https://github.com/3scale/3scale-go-client
...,...,...,...,...,...,...,...
10956,zerodayz,25482,sosreport-operator,30872,andreaskaris/sosreport-operator,0.0,https://github.com/zerodayz/sosreport-operator
10957,zerodayz,25482,talos,30863,siderolabs/talos,0.0,https://github.com/zerodayz/talos
10958,zerodayz,25482,tests,30838,kata-containers/tests,0.0,https://github.com/zerodayz/tests
10959,zerodayz,25482,web,30839,openshifttips/web,0.0,https://github.com/zerodayz/web


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
repo_git          object
dtype: object

In [4]:
x = pd.DataFrame()
repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT 
                r.repo_name,
                r.repo_id,
                r.repo_git,
                r.url,
                c.cmt_author_name,
                pc.pr_cmt_author_email
            FROM
                repo r, commits c, pull_request_commits pc
            WHERE
                r.repo_id = c.repo_id AND
                c.repo_id = pc.repo_id
            ORDER BY
                repo_name
            LIMIT 100;
    """)
x = pd.read_sql(repo_query, con=engine)
display(x)
x.dtypes

,repo_name,repo_id,repo_git,url,cmt_author_name,pr_cmt_author_email
0,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
1,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
2,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
3,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
4,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
...,...,...,...,...,...,...
95,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
96,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
97,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None
98,8docs,25535,https://github.com/CentOS/8docs,None,Petr Bokoc,None


repo_name              object
repo_id                 int64
repo_git               object
url                    object
cmt_author_name        object
pr_cmt_author_email    object
dtype: object

In [5]:
x = pd.DataFrame()
repo_query = salc.sql.text(f"""
             SET SCHEMA 'augur_data';
             SELECT 
                r.repo_name,
                r.repo_id,
                r.repo_git,
                r.url,
                c.cmt_author_name
            FROM
                repo r, commits c
            WHERE
                r.repo_id = c.repo_id
            ORDER BY
                repo_name
            LIMIT 100;
    """)
x = pd.read_sql(repo_query, con=engine)
display(x)
x.dtypes

,repo_name,repo_id,repo_git,url,cmt_author_name
0,11bot,27924,https://github.com/coreos/11bot,None,Benjamin Gilbert
1,11bot,27924,https://github.com/coreos/11bot,None,Benjamin Gilbert
2,11bot,27924,https://github.com/coreos/11bot,None,Benjamin Gilbert
3,11bot,27924,https://github.com/coreos/11bot,None,Benjamin Gilbert
4,11bot,27924,https://github.com/coreos/11bot,None,Benjamin Gilbert
...,...,...,...,...,...
95,3scale,33134,https://github.com/3scale/3scale.github.io,None,Thomas Maas
96,3scale,33134,https://github.com/3scale/3scale.github.io,None,Thomas Maas
97,3scale,33134,https://github.com/3scale/3scale.github.io,None,Thomas Maas
98,3scale,33134,https://github.com/3scale/3scale.github.io,None,Thomas Maas


repo_name          object
repo_id             int64
repo_git           object
url                object
cmt_author_name    object
dtype: object

In [23]:
x = pd.DataFrame()
repo_query = salc.sql.text(f"""
            SET SCHEMA 'augur_data';
            SELECT r.repo_id,
            r.repo_git,
            prm.cntrb_id,
            ca.cntrb_id,
            i.cntrb_id,
            prr.cntrb_id
            FROM
            pull_request_meta prm, repo r, commits c, contributors_aliases ca, issues i, pull_request_reviews prr
             WHERE
            r.repo_id = prm.repo_id and
            c.repo_id = r.repo_id and
            i.repo_id = r.repo_id and
            prr.repo_id = r.repo_id and
            c.cmt_committer_email = ca.alias_email

            limit 5000;
    """)
x = pd.read_sql(repo_query, con=engine)
x.columns =['Repo ID', 'Git', 'PR Creators', 'Commit Contributors', 'Issue Authors', 'PR Reviewers']

display(x)
x.dtypes

,Repo ID,Git,PR Creators,Commit Contributors,Issue Authors,PR Reviewers
0,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25440
1,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25451
2,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25446
3,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25446
4,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25435
...,...,...,...,...,...,...
4995,24442,https://github.com/operate-first/blueprint,25451,25445,25449,25443
4996,24442,https://github.com/operate-first/blueprint,25451,25445,25449,25443
4997,24442,https://github.com/operate-first/blueprint,25451,25445,25449,25443
4998,24442,https://github.com/operate-first/blueprint,25451,25445,25449,25444


Repo ID                 int64
Git                    object
PR Creators             int64
Commit Contributors     int64
Issue Authors           int64
PR Reviewers            int64
dtype: object

1. drop duplicates, unique row values
2. transpose the contributor ids to columns and get counts
3. pass this to the networkx graph

In [24]:
set(x['Git'])

{'https://github.com/operate-first/blueprint'}

In [ ]:
[repo, (set of contributor IDs for repo),
repo , (set of )]

In [20]:
x.head(20)

,Repo ID,Git,PR Creators,Commit Contributors,Issue Authors,PR Reviewers
0,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25440
1,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25451
2,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25446
3,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25446
4,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25435
5,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25439
6,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25443
7,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25446
8,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25443
9,24442,https://github.com/operate-first/blueprint,25451,26033,25449,25439
